**07-01 순환 신경망**

In [ ]:
#hidden_state_t = 0
#for input_t in input_length:
#    output_t = tanh(input_t, hidden_state_t)
#    hidden_state_t = output_t

In [ ]:
import numpy as np

timesteps = 10
input_size = 4
hidden_size = 8

inputs = np.random.random((timesteps, input_size))

hidden_state_t = np.zeros((hidden_size,))

In [ ]:
print(hidden_state_t)

[0. 0. 0. 0. 0. 0. 0. 0.]


In [ ]:
Wx = np.random.random((hidden_size, input_size))
Wh = np.random.random((hidden_size, hidden_size))
b = np.random.random((hidden_size,))

In [ ]:
print(np.shape(Wx))
print(np.shape(Wh))
print(np.shape(b))

(8, 4)
(8, 8)
(8,)


In [ ]:
total_hidden_states = []

for input_t in inputs:
    output_t = np.tanh(np.dot(Wx, input_t) + np.dot(Wh, hidden_state_t) + b)
    total_hidden_states.append(list(output_t))
    print(np.shape(total_hidden_states))
    hidden_state_t = output_t

total_hidden_states = np.stack(total_hidden_states, axis = 0)

print(total_hidden_states)

In [ ]:
import torch
import torch.nn as nn

input_size = 5
hidden_size = 8

inputs = torch.Tensor(1,10,5)

cell = nn.RNN(input_size, hidden_size, batch_first=True)

outputs, _status = cell(inputs)

In [ ]:
print(outputs.shape)

torch.Size([1, 10, 8])


In [ ]:
print(_status.shape)

torch.Size([1, 1, 8])


In [ ]:
inputs = torch.Tensor(1,10,5)

In [ ]:
cell=nn.RNN(input_size = 5, hidden_size = 8, num_layers = 2, batch_first=True)
outputs, _status = cell(inputs)

In [ ]:
print(outputs.shape)

torch.Size([1, 10, 8])


In [ ]:
print(_status.shape)

torch.Size([2, 1, 8])


In [ ]:
import torch
import torch.nn as nn

input_size = 5
hidden_size = 8

inputs = torch.Tensor(1,10,5)

cell = nn.RNN(input_size, hidden_size,num_layers=2, batch_first=True, bidirectional= True)

outputs, _status = cell(inputs)

In [ ]:
print(outputs.shape)
print(_status.shape)

torch.Size([1, 10, 16])
torch.Size([4, 1, 8])


**07-04 문자단위 RNN**

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

input_str = 'apple'
label_str = 'pple!'
char_vocab = sorted(list(set(input_str+label_str)))
vocab_size = len(char_vocab)
print('문자 집합의 크기: {}'.format(vocab_size))

문자 집합의 크기: 5


In [ ]:
input_size = vocab_size
hidden_size = 5
output_size = 5
learning_rate = 0.1

char_to_index = dict((c,i) for i,c in enumerate(char_vocab))
print(char_to_index)

index_to_char={}
for key, value in char_to_index.items():
    index_to_char[value] = key
print(index_to_char)

{'!': 0, 'a': 1, 'e': 2, 'l': 3, 'p': 4}
{0: '!', 1: 'a', 2: 'e', 3: 'l', 4: 'p'}


In [ ]:
x_data = [char_to_index[c] for c in input_str]
y_data = [char_to_index[c] for c in label_str]
print(x_data)
print(y_data)

[1, 4, 4, 3, 2]
[4, 4, 3, 2, 0]


In [ ]:
x_data = [x_data]
y_data = [y_data]


In [ ]:
x_one_hot = [np.eye(vocab_size)[x] for x in x_data]
print(x_one_hot)

[array([[0., 1., 0., 0., 0.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 0., 1.],
       [0., 0., 0., 1., 0.],
       [0., 0., 1., 0., 0.]])]


In [ ]:
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([1, 5, 5])
레이블의 크기: torch.Size([1, 5])


In [ ]:
class Net(torch.nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_size, hidden_size, batch_first=True)
        self.fc = torch.nn.Linear(hidden_size, output_size, bias = True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(x)
        return x

In [ ]:
net = Net(input_size, hidden_size, output_size)
outputs = net(X)
print(outputs.shape)

torch.Size([1, 5, 5])


In [ ]:
print(outputs.view(-1,input_size).shape)

torch.Size([5, 5])


In [ ]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([1, 5])
torch.Size([5])


In [ ]:
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [ ]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1,input_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    result = outputs.data.numpy().argmax(axis = 2)
    result_str =''.join([index_to_char[c] for c in np.squeeze(result)])
    print(i,'loss: ', loss.item(),'prediction: ', result, "true Y: ", y_data, "prediction str: ", result_str)

**07-05 문자 단위 RNN**

In [7]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

sentence = ("if you want to build a ship, don't drum up people together to "
            "collect wood and don't assign them tasks and work, but rather "
            "teach them to long for the endless immensity of the sea.")

char_set = list(set(sentence))
char_dic = {c: i for i,c in enumerate(char_set)}
print(char_dic)

{'g': 0, 'm': 1, 'y': 2, 'f': 3, 'u': 4, 'e': 5, 'i': 6, 'w': 7, 'c': 8, 't': 9, 'r': 10, 's': 11, ',': 12, 'o': 13, 'l': 14, "'": 15, ' ': 16, 'a': 17, 'd': 18, 'n': 19, 'h': 20, '.': 21, 'k': 22, 'b': 23, 'p': 24}


In [2]:
dic_size = len(char_dic)
print('문자 집합의 크기 : {}'.format(dic_size))

문자 집합의 크기 : 25


In [4]:
hidden_size = dic_size
sequence_length = 10
learning_rate = 0.1

x_data = []
y_data = []

for i in range(0,len(sentence) - sequence_length):
    x_str = sentence[i:i + sequence_length]
    y_str = sentence[i +1: i + sequence_length +1]
    print(i, x_str, '->',y_str)

    x_data.append([char_dic[c] for c in x_str])
    y_data.append([char_dic[c] for c in y_str])

0 if you wan -> f you want
1 f you want ->  you want 
2  you want  -> you want t
3 you want t -> ou want to
4 ou want to -> u want to 
5 u want to  ->  want to b
6  want to b -> want to bu
7 want to bu -> ant to bui
8 ant to bui -> nt to buil
9 nt to buil -> t to build
10 t to build ->  to build 
11  to build  -> to build a
12 to build a -> o build a 
13 o build a  ->  build a s
14  build a s -> build a sh
15 build a sh -> uild a shi
16 uild a shi -> ild a ship
17 ild a ship -> ld a ship,
18 ld a ship, -> d a ship, 
19 d a ship,  ->  a ship, d
20  a ship, d -> a ship, do
21 a ship, do ->  ship, don
22  ship, don -> ship, don'
23 ship, don' -> hip, don't
24 hip, don't -> ip, don't 
25 ip, don't  -> p, don't d
26 p, don't d -> , don't dr
27 , don't dr ->  don't dru
28  don't dru -> don't drum
29 don't drum -> on't drum 
30 on't drum  -> n't drum u
31 n't drum u -> 't drum up
32 't drum up -> t drum up 
33 t drum up  ->  drum up p
34  drum up p -> drum up pe
35 drum up pe -> rum up peo
36

In [5]:
print(x_data[0])
print(y_data[0])

[6, 3, 16, 2, 13, 4, 16, 7, 17, 19]
[3, 16, 2, 13, 4, 16, 7, 17, 19, 9]


In [9]:
x_one_hot = [np.eye(dic_size)[x] for x in x_data]
X = torch.FloatTensor(x_one_hot)
Y = torch.LongTensor(y_data)

In [10]:
print('훈련 데이터의 크기: {}'.format(X.shape))
print('레이블의 크기: {}'.format(Y.shape))

훈련 데이터의 크기: torch.Size([170, 10, 25])
레이블의 크기: torch.Size([170, 10])


In [11]:
print(X[0])

tensor([[0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,

In [12]:
print(Y[0])

tensor([ 3, 16,  2, 13,  4, 16,  7, 17, 19,  9])


In [13]:
class Net(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, layers):
        super(Net, self).__init__()
        self.rnn = torch.nn.RNN(input_dim, hidden_dim, num_layers = layers, batch_first=True)
        self.fc = torch.nn.Linear(hidden_dim, hidden_dim, bias = True)

    def forward(self, x):
        x, _status = self.rnn(x)
        x = self.fc(X)
        return X

net = Net(dic_size, hidden_size, 2)

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), learning_rate)

In [14]:
outputs = net(X)
print(outputs.shape)

torch.Size([170, 10, 25])


In [15]:
print(Y.shape)
print(Y.view(-1).shape)

torch.Size([170, 10])
torch.Size([1700])


In [1]:
for i in range(100):
    optimizer.zero_grad()
    outputs = net(X)
    loss = criterion(outputs.view(-1, dic_size), Y.view(-1))
    loss.backward()
    optimizer.step()

    results = outputs.argmax(dim=2)
    predict_str = ""
    for j, result in enumerate(results):
        if j == 0:
            predict_str += ''.join([char_set[t] for t in result])
        else:
            predict_str += char_set[result[-1]]
    print(predict_str)

NameError: name 'optimizer' is not defined